In [1]:
#-*- coding: utf-8 -*-
#import sys 
#reload(sys)
#sys.setdefaultencoding('utf-8')

import requests
from pprint import pprint


In [2]:
# kakao api
APP_KEY = "${API_KEY}"
kakao_url = "https://dapi.kakao.com/v2/local/search/address.json"
kakao_headers = {"Authorization" : "KakaoAK {}".format(APP_KEY)}

# naver api
client_id = "${CLIENT_ID}"
client_secret = "${CLIENT_SECRET}"
naver_url = "https://openapi.naver.com/v1/map/geocode" 
naver_headers = {"X-Naver-Client-Id" : "{}".format(client_id) , "X-Naver-Client-Secret": "{}".format(client_secret)}

In [3]:
def kakao_req(params, method):
#     print('HTTP Method: %s' % method)
#     print('Request URL: %s' % kakao_url)
#     print('Headers: %s' % kakao_headers)
#     print('QueryString: %s' % params)

    if method == 'GET':
        return requests.get(kakao_url, headers=kakao_headers, params=params)
    
def naver_req(params, method):
#     print('HTTP Method: %s' % method)
#     print('Request URL: %s' % naver_url)
#     print('Headers: %s' % naver_headers)
#     print('QueryString: %s' % params)
    
    if method == 'GET':
        return requests.get(naver_url, headers=naver_headers, params=params)

In [4]:
def get_location(address):

    params = {"query": "{}".format(address)}

    response = naver_req(params, 'GET')
        

#    #kakao parsing
#     pprint(data['meta']['total_count'])
#     pprint(data["documents"][0]["x"])
#     pprint(data["documents"][0]["y"])
    
        
    return response

In [5]:
import findspark
findspark.init()
import pyspark
import os
import numpy as np

In [6]:
#sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [7]:
basic_folder = '/home/project/real_estate/data/office_rent_data/'

file_list = os.listdir(basic_folder)
path_list=[]
for file in file_list:
    print(file)
    path_list.append(basic_folder + file)
    
print(path_list)

office_rent_201312.csv
office_rent_201802.csv
office_rent_201410.csv
office_rent_201210.csv
office_rent_201109.csv
office_rent_201302.csv
office_rent_201602.csv
office_rent_201607.csv
office_rent_201505.csv
office_rent_201407.csv
office_rent_201104.csv
office_rent_201403.csv
office_rent_201503.csv
office_rent_201107.csv
office_rent_201611.csv
office_rent_201806.csv
office_rent_201702.csv
office_rent_201509.csv
office_rent_201103.csv
office_rent_201710.csv
office_rent_201101.csv
office_rent_201405.csv
office_rent_201801.csv
office_rent_201310.csv
office_rent_201501.csv
office_rent_201510.csv
office_rent_201409.csv
office_rent_201106.csv
office_rent_201804.csv
office_rent_201201.csv
office_rent_201601.csv
office_rent_201603.csv
office_rent_201707.csv
office_rent_201507.csv
office_rent_201610.csv
office_rent_201307.csv
office_rent_201108.csv
office_rent_201412.csv
office_rent_201411.csv
office_rent_201709.csv
office_rent_201202.csv
office_rent_201605.csv
office_rent_2017012.csv
office_ren

In [8]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("header", "true").option("inferSchema", "true").load(path_list)

In [9]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- build_year: integer (nullable = true)
 |-- deposit: string (nullable = true)
 |-- rental_fee: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- city: string (nullable = true)
 |-- sub_city: string (nullable = true)
 |-- dong: string (nullable = true)
 |-- code: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- floor: integer (nullable = true)
 |-- exclusive_private_area: double (nullable = true)
 |-- address: string (nullable = true)
 |-- detailed_address: string (nullable = true)



In [10]:
df.show()
df.count()

+---+----------+-------+----------+----+-----+-----+-----+--------+----+-----+------------+-----+----------------------+--------------+----------------+
|_c0|build_year|deposit|rental_fee|year|month|  day| city|sub_city|dong| code|        name|floor|exclusive_private_area|       address|detailed_address|
+---+----------+-------+----------+----+-----+-----+-----+--------+----+-----+------------+-----+----------------------+--------------+----------------+
|  0|      2004| 17,000|         0|2018|    1| 1~10|서울특별시|     종로구| 내수동|11110|       광화문시대|   16|                 48.86|서울특별시 종로구  내수동|              74|
|  1|      2004| 25,000|         0|2018|    1| 1~10|서울특별시|     종로구| 내수동|11110| 경희궁의 아침 3단지|   10|                  37.5|서울특별시 종로구  내수동|              72|
|  2|      2004|  1,000|        90|2018|    1| 1~10|서울특별시|     종로구| 내수동|11110|       용비어천가|   16|                 37.56|서울특별시 종로구  내수동|              75|
|  3|      2004| 22,000|         0|2018|    1| 1~10|서울특별시|     종로구| 내수동|11110|    

390185

In [11]:
#filter_df = df.filter((df["detailed_address"] == "") | df["detailed_address"].isNull() | isnan(df["detailed_address"]))
#filter_df.show()
#filter_df.count() == 36

filter_df = df.na.drop()



In [12]:
from pyspark.sql.functions import *

In [13]:
unique_date_df = filter_df.groupBy(filter_df.year,filter_df.month, filter_df.day).agg(count("day").alias('count'))

unique_date_list = unique_date_df.collect()

new_date_list = []

for unique_date in unique_date_list:
    if unique_date[1] < 10 :
        if int(unique_date[2].split('~')[0]) < 10 : 
            date = str(unique_date[0]) + '-' + '0' + str(unique_date[1]) + '-' + '0' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
        else:
            date = str(unique_date[0]) + '-' + '0' + str(unique_date[1]) + '-' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
        
    else:
        if int(unique_date[2].split('~')[0]) < 10 : 
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' + '0' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
    
        else:
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
    
    
    new_date_list.append([unique_date[0], unique_date[1],unique_date[2],date])
    
    
unique_date_df = sc.parallelize(new_date_list).toDF(['year','month','day','date'])

In [14]:
added_date_df = filter_df.join(unique_date_df, ['year','month','day']).sort('date')
added_date_df.show()
added_date_df.count()

+----+-----+----+---+----------+-------+----------+-----+--------+------+-----+------------+-----+----------------------+----------------+----------------+-------------------+
|year|month| day|_c0|build_year|deposit|rental_fee| city|sub_city|  dong| code|        name|floor|exclusive_private_area|         address|detailed_address|               date|
+----+-----+----+---+----------+-------+----------+-----+--------+------+-----+------------+-----+----------------------+----------------+----------------+-------------------+
|2011|    1|1~10| 23|      2005|    500|        65|서울특별시|     성동구| 성수동1가|11200|       뚝섬리버빌|   10|                 26.73|서울특별시 성동구  성수동1가|          13-117|2011-01-01T00:00:00|
|2011|    1|1~10|  9|      1998|  1,000|        65|서울특별시|     성동구|   도선동|11200|       신한넥스텔|    6|                 44.86|  서울특별시 성동구  도선동|              14|2011-01-01T00:00:00|
|2011|    1|1~10| 22|      2004| 10,000|         0|서울특별시|     성동구| 성수동1가|11200|      성수 그린빌|    4|                 34.59

380095

In [15]:
new_df = added_date_df.groupBy(added_date_df.address,added_date_df.detailed_address).agg(count("detailed_address").alias("count"))
new_df.show()
new_df.count()

+-----------------+----------------+-----+
|          address|detailed_address|count|
+-----------------+----------------+-----+
|   대전광역시 유성구  지족동|           863-2|  164|
|  경기도 평택시 안중읍 안중리|          252-33|   36|
|  경기도 화성시 남양읍 남양리|          1265-3|    5|
|   부산광역시 금정구  장전동|          364-24|   10|
|   부산광역시 금정구  장전동|          227-12|   29|
|  경기도 안산시 상록구  사동|          1197-4|   83|
|     경기도 이천시  중리동|           211-8|    6|
|경상남도 창원시 성산구  상남동|            11-6|    2|
|    인천광역시 남구  도화동|            90-2|   15|
|  서울특별시 동대문구  이문동|          325-20|   65|
|부산광역시 기장군 기장읍 대라리|           500-6|   13|
|  서울특별시 서대문구  창천동|           29-81|   35|
|   서울특별시 강북구  수유동|            48-1|  175|
|   서울특별시 강동구  암사동|           502-3|   16|
| 경기도 수원시 영통구  이의동|            1330|  364|
|   대구광역시 달서구  두류동|           150-9|    5|
|   부산광역시 금정구  구서동|          192-11|   13|
|  부산광역시 부산진구  부전동|           414-2|   20|
|   인천광역시 남동구  만수동|             867|    3|
|   서울특별시 은평구  불광동|           319-8|   14|
+----------

9480

In [16]:
sum = new_df.select(sum("count"))

In [17]:
sum.show()

+----------+
|sum(count)|
+----------+
|    380095|
+----------+



In [18]:
unique_address_list = new_df.collect()
print unique_address_list[0][0]

대전광역시 유성구  지족동


In [19]:
location_list = []

# checking why no result data
# response = get_location("경상북도 영천시 신령면 완전리") -> 신녕면
# print response.status_code
# data = response.json()
# print data

index = 1.0;
length = len(unique_address_list)
count = 0;

print length 

for address in unique_address_list:    
    full_address = (address[0] + " " + address[1]).encode('utf8')
    #print str("full " + full_address)
    response = get_location(full_address)

    if response.status_code == 200 :
        
        data = response.json()
        
        
        x = data['result']['items'][0]['point']['x']
        y = data['result']['items'][0]['point']['y']
    
        location_list.append([address[0],address[1],y,x])
        
    else: 
        print str("error address: " + full_address)
        count += 1
        print count
        
        location_list.append([address[0],address[1] ,np.nan,np.nan])
    
    if(index % 100 == 0):
        print ("%0.2f" % (index / length * 100.0)) + "% completed"
        
    index += 1.0

     

9480
1.05% completed
2.11% completed
3.16% completed
4.22% completed
5.27% completed
6.33% completed
7.38% completed
8.44% completed
9.49% completed
error address: 부산광역시 금정구  장전동 74-93
1
10.55% completed
11.60% completed
error address: 서울특별시 관악구  봉천동 863-6
2
12.66% completed
13.71% completed
14.77% completed
15.82% completed
16.88% completed
17.93% completed
18.99% completed
20.04% completed
21.10% completed
22.15% completed
23.21% completed
24.26% completed
25.32% completed
26.37% completed
27.43% completed
28.48% completed
29.54% completed
30.59% completed
31.65% completed
32.70% completed
33.76% completed
34.81% completed
35.86% completed
36.92% completed
37.97% completed
39.03% completed
40.08% completed
41.14% completed
42.19% completed
43.25% completed
44.30% completed
45.36% completed
46.41% completed
47.47% completed
48.52% completed
49.58% completed
50.63% completed
51.69% completed
52.74% completed
53.80% completed
54.85% completed
55.91% completed
56.96% completed
58.02% com

In [20]:
#print location_list[0]


#checking_df = df.filter(df.address == "인천광역시 서구  왕길동").filter(df.detailed_address == "가-81-1")

#checking_df.show()
#print(checking_df.count())

                
location = sc.parallelize(location_list).toDF(['address','detailed_address','latitude','longitude'])
location.show()

+-----------------+----------------+----------+-----------+
|          address|detailed_address|  latitude|  longitude|
+-----------------+----------------+----------+-----------+
|   대전광역시 유성구  지족동|           863-2|36.3825628|127.3205555|
|  경기도 평택시 안중읍 안중리|          252-33|36.9881887|126.9255043|
|  경기도 화성시 남양읍 남양리|          1265-3|37.2101135|126.8181706|
|   부산광역시 금정구  장전동|          364-24|35.2349315|129.0886999|
|   부산광역시 금정구  장전동|          227-12|35.2372391| 129.087994|
|  경기도 안산시 상록구  사동|          1197-4|37.3017055|126.8511012|
|     경기도 이천시  중리동|           211-8|37.2784762| 127.444716|
|경상남도 창원시 성산구  상남동|            11-6| 35.222296|128.6851114|
|    인천광역시 남구  도화동|            90-2|37.4677795|126.6593777|
|  서울특별시 동대문구  이문동|          325-20|37.5944677|127.0593227|
|부산광역시 기장군 기장읍 대라리|           500-6|35.2424389|129.2117922|
|  서울특별시 서대문구  창천동|           29-81| 37.556057|126.9382413|
|   서울특별시 강북구  수유동|            48-1|37.6344091|127.0228738|
|   서울특별시 강동구  암사동|           502-3|37.5

In [21]:
location.count()

9480

In [22]:
# error 
#new_df_id = new_df.withColumn('id', monotonically_increasing_id())
#new_df_id.show()
#new_df_id.count()
#new_df_id.filter(new_df_id.id == '10000').show()
#add_df.count()


#location_id = location.withColumn('id', row_number().over (Window.partitionBy().partitionBy(location['logitude']).orderBy(lit(1))))
#location_id.show()
#location_id.count()

#location_id.filter(location_id.id == '37777').show()


#add_df = new_df_id.select('address', 'detailed_address','id').join(location_id , new_df_id.id == location_id.id).drop(new_df_id.id)
#add_df.orderBy('id', ascending=True).show()
#add_df.count()

In [23]:
joined_df = added_date_df.join(location.select('address','detailed_address','latitude','longitude'), ['address','detailed_address'])

In [24]:
joined_df.show()
joined_df.count()

+----------------+----------------+----+-----+-----+---+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|         address|detailed_address|year|month|  day|_c0|build_year|deposit|rental_fee|city|sub_city|dong| code|        name|floor|exclusive_private_area|               date|  latitude|  longitude|
+----------------+----------------+----+-----+-----+---+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|경기도 수원시 영통구  이의동|            1330|2012|   11| 1~10|  1|      2012|    500|        55| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|    7|                 28.96|2012-11-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이의동|            1330|2013|    1| 1~10|  1|      2012|  7,000|         0| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|   12|                 28.96|2013-01-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이

380095

In [25]:
drop_index_df = joined_df.drop("_c0")
drop_index_df.show()
drop_index_df.count()

+----------------+----------------+----+-----+-----+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|         address|detailed_address|year|month|  day|build_year|deposit|rental_fee|city|sub_city|dong| code|        name|floor|exclusive_private_area|               date|  latitude|  longitude|
+----------------+----------------+----+-----+-----+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|경기도 수원시 영통구  이의동|            1330|2012|   11| 1~10|      2012|    500|        55| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|    7|                 28.96|2012-11-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이의동|            1330|2013|    1| 1~10|      2012|  7,000|         0| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|   12|                 28.96|2013-01-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이의동|            1330|

380095

In [26]:
final_office_df = drop_index_df.na.drop()
final_office_df = final_office_df.withColumn('deposit',regexp_replace('deposit',"\\,", "")).withColumn('rental_fee',regexp_replace('rental_fee',"\\,", ""))
final_office_df.show()
final_office_df.count()

+----------------+----------------+----+-----+-----+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|         address|detailed_address|year|month|  day|build_year|deposit|rental_fee|city|sub_city|dong| code|        name|floor|exclusive_private_area|               date|  latitude|  longitude|
+----------------+----------------+----+-----+-----+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|경기도 수원시 영통구  이의동|            1330|2012|   11| 1~10|      2012|    500|        55| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|    7|                 28.96|2012-11-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이의동|            1330|2013|    1| 1~10|      2012|   7000|         0| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|   12|                 28.96|2013-01-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이의동|            1330|

379966

In [28]:
final_office_df.write.csv("final_office_rent")